In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
from ipywidgets import *  # for interactive test plot
import numba
from numba import jit
from numba import njit
import time

In [2]:
from scipy.interpolate import griddata

In [3]:

from matplotlib.animation import FFMpegWriter
import matplotlib.animation as manimation

In [4]:
%matplotlib notebook

In [5]:
import sys
sys.path.append('../heat_lamps')

In [6]:
import 
from warm_lpm5 import panels
from warm_lpm5 import field_functions as field
from warm_lpm5 import time_step_functions as step

ModuleNotFoundError: No module named 'warm_lpm5'

In [54]:
from importlib import reload


panels = reload(warm_lpm5.panels)
field = reload(warm_lpm5.field_functions)
step = reload(warm_lpm5.time_step_functions)

In [59]:

@njit
def f0(x,v,L,vth, amp):
    """
    """
    return 1./ np.sqrt(2.*np.pi) /vth * np.exp(-v**2/2./vth**2) * (1+amp *np.cos(2*np.pi/L*x))

@njit
def f_M(x,v,L,vth):
    """
    """
    return 1./ np.sqrt(2.*np.pi) /vth * np.exp(-v**2/2./vth**2)


L = 2 * 2*np.pi
k = 2*np.pi / L
vth = 1.
amp = .01
q = -1.
m = 1.
qm = q/m
xlim = (0,L)
npanels_x = 128
vlim = (-5,5)
npanels_v = 128
npanels = npanels_x * npanels_v

Elim = (-1.5*amp/k, 1.5*amp/k)

delta = 0.
interpolation = 'cubic'
do_remesh = False
remesh_freq = 0
if do_remesh:
    remesh_string = ''
else:
    remesh_string = 'no'

dt = .01
num_steps = 4000
tf = dt * num_steps


movie_freq = 5
diag_freq = 1
dump_freq = min(movie_freq,diag_freq)


sim_group = 'delf_LD_noremesh_%.03f'%delta
sim_name = 'LD_remeshing_%i_steps_interp_%s_linear_standardcase_'%(remesh_freq,interpolation) +        'L_%.03f_vth_%.03f_amp_%.03f_'%(L,vth,amp) +        'npanels_x_%i_npanels_v_%i_'%(npanels_x,npanels_v) +        'vmax_%.03f_delta_%.03f_'%(vlim[1],delta) +        'dt_%.03f_num_steps_%i_'%(dt,num_steps) +        'tf_%.03f'%(tf)
sim_title = 'LD with %s re-meshing every %i steps, interpolation: %s,\n'%(remesh_string, remesh_freq,interpolation) +        'L %.03f, vth %.03f, amp %.03f,\n'%(L,vth,amp) +        'npanels_x %i, npanels_v %i, '%(npanels_x,npanels_v) +        'vmax %.03f, delta %.03f,\n'%(vlim[1],delta) +        'dt %.03f, nt %i, '%(dt,num_steps) +        'tf %.03f'%(tf)

simulations_dir = 'simulations/'
sim_group_dir = simulations_dir + sim_group + '/'
sim_dir = sim_group_dir + sim_name + '/'
params_file = sim_dir + 'sim_params.txt'

if not os.path.exists(simulations_dir):
    os.makedirs(simulations_dir)
if not os.path.exists(sim_group_dir):
    os.makedirs(sim_group_dir)
if not os.path.exists(sim_dir):
    os.makedirs(sim_dir)

species_data = panels.Panels()
species_data.L = L
species_data.vth = vth
species_data.q = q
species_data.m = m
species_data.qm = qm
species_data.set_xs_vs(npanels_x,xlim,npanels_v,vlim)
species_data.set_panels()
species_data.set_f0_weights(lambda x,v: f0(x,v,L,vth,amp))
if species_data.npanels <= 3000:
    species_data.field_obj = field.exact_field(delta)
else:
    species_data.field_obj = field.sort_field(delta)

panels.Panels.initialize = step.initialize_leapfrog
panels.Panels.update = step.update_leapfrog
panels.Panels.diag_dump = step.diag_dump_leapfrog

species_data.setup_diagnostic_dir(sim_dir)

# write cell contents to file
with open(params_file,'w') as cell_file:
    cell_file.write(In[-1])

In [ ]:
%%time
species_data.initialize(dt)

%run -i 'run_sim.py'
%run -i 'plot_stuff.py'

1/11/2019
Trying to modularize this code and having problems
Problem
1. Can't make any ide work properly

Calling module from notebook
* Don't know how to call Python script properly
* Panel movie doesn't work right

# for running batches:
batch = one group, one form of simulation

batch script should
* create batch directory
* have paramater set
* for each combination of parameters
    * generate simulation directory
    * generate a parameter record
    * run a simulation corresponding to that parameter combination
    * generate log file for that run


sim dir
* parameter_file
* run log
    * version of code run
    * success of run
* output_directory
* analysis_directory